In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
os.getcwd()

'/home/anproulx/course-materials-2020/lectures/14-may/02-intro-to-stats'

In [6]:
os.chdir("/home/anproulx/Proulx-Andr-anne-QLSC612/practical")

In [7]:
data=pd.read_csv("brainsize.csv", delimiter=";", na_values='.')
data=data.set_index("Unnamed: 0")
data=data.drop("MRI_Count", axis=1)

data.head()
#print(len(data))

,Gender,FSIQ,VIQ,PIQ,Weight,Height
Unnamed: 0,,,,,,
1,Female,133,132,124,118.0,64.5
2,Male,140,150,124,NaN,72.5
3,Male,139,123,150,143.0,73.3
4,Male,133,129,128,172.0,68.8
5,Female,137,132,134,147.0,65.0


In [8]:
#Create variable of random noise

np.random.seed(0)
partY= np.random.normal(0,1,40)

#Add to existing dataframe
data["partY"]=partY
print(data)

            Gender  FSIQ  VIQ  PIQ  Weight  Height     partY
Unnamed: 0                                                  
1           Female   133  132  124   118.0    64.5  1.764052
2             Male   140  150  124     NaN    72.5  0.400157
3             Male   139  123  150   143.0    73.3  0.978738
4             Male   133  129  128   172.0    68.8  2.240893
5           Female   137  132  134   147.0    65.0  1.867558
6           Female    99   90  110   146.0    69.0 -0.977278
7           Female   138  136  131   138.0    64.5  0.950088
8           Female    92   90   98   175.0    66.0 -0.151357
9             Male    89   93   84   134.0    66.3 -0.103219
10            Male   133  114  147   172.0    68.8  0.410599
11          Female   132  129  124   118.0    64.5  0.144044
12            Male   141  150  128   151.0    70.0  1.454274
13            Male   135  129  124   155.0    69.0  0.761038
14          Female   140  120  147   155.0    70.5  0.121675
15          Female    96

In [7]:
#generate predictions of the partY variable until finds significant results
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

In [157]:
#get different combinations of feature variables
combination=[]

from itertools import combinations
comb= set(combinations(data.drop("partY", axis=1) , 4))

for val in comb:
    combination.append(val)
    
print(combination)

[('Gender', 'FSIQ', 'PIQ', 'Height'), ('FSIQ', 'VIQ', 'Weight', 'Height'), ('Gender', 'VIQ', 'Weight', 'Height'), ('Gender', 'FSIQ', 'VIQ', 'PIQ'), ('Gender', 'FSIQ', 'VIQ', 'Height'), ('Gender', 'FSIQ', 'Weight', 'Height'), ('Gender', 'PIQ', 'Weight', 'Height'), ('Gender', 'VIQ', 'PIQ', 'Height'), ('FSIQ', 'VIQ', 'PIQ', 'Height'), ('FSIQ', 'PIQ', 'Weight', 'Height'), ('Gender', 'FSIQ', 'PIQ', 'Weight'), ('VIQ', 'PIQ', 'Weight', 'Height'), ('Gender', 'VIQ', 'PIQ', 'Weight'), ('Gender', 'FSIQ', 'VIQ', 'Weight'), ('FSIQ', 'VIQ', 'PIQ', 'Weight')]


In [175]:
#test model with the different combinations of feature variables
list_significant=[]
list_ind=[]

#loop through the combination lis
for val in combination:
    name_var1=str(val[0])
    name_var2=str(val[1])
    name_var3=str(val[2])
    name_var4=str(val[3])
    
    feat_1=data[name_var1]
    feat_2=data[name_var2]
    feat_3=data[name_var3]
    feat_4=data[name_var4]
    
    # Fit the model with the x variable
    model = ols("partY ~ feat_1 + feat_2 + feat_3 + feat_4", data).fit()
    
    print(val)
    print(model.summary())
    
    pval=model.pvalues
    
    # Add results <0.05 to list_significant
    for index, value in pval.items():
        if value <= 0.05:
            ind_sig=index
            
            list_ind.append(index)
            list_significant.append(val)
            
            break
            
#report only the models with p<0.05
    
print(list_significant)
print(list_ind)

('Gender', 'FSIQ', 'PIQ', 'Height')
                            OLS Regression Results                            
Dep. Variable:                  partY   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     2.689
Date:                Mon, 18 May 2020   Prob (F-statistic):             0.0475
Time:                        17:45:47   Log-Likelihood:                -48.895
No. Observations:                  39   AIC:                             107.8
Df Residuals:                      34   BIC:                             116.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercep

In [176]:
for ind in range (len(list_significant)):
    if list_ind[ind]!="Intercept":
        print(list_significant[ind], list_ind[ind])

('Gender', 'FSIQ', 'VIQ', 'PIQ') feat_1[T.Male]
('FSIQ', 'VIQ', 'PIQ', 'Weight') feat_4


In [171]:
#partY 2
np.random.seed(2)
partY2= np.random.normal(0,1,40)
data["partY2"]=partY2
data

,Gender,FSIQ,VIQ,PIQ,Weight,Height,partY,partY2
Unnamed: 0,,,,,,,,
1,Female,133,132,124,118.0,64.5,1.764052,-0.416758
2,Male,140,150,124,NaN,72.5,0.400157,-0.056267
3,Male,139,123,150,143.0,73.3,0.978738,-2.136196
4,Male,133,129,128,172.0,68.8,2.240893,1.640271
5,Female,137,132,134,147.0,65.0,1.867558,-1.793436
6,Female,99,90,110,146.0,69.0,-0.977278,-0.841747
7,Female,138,136,131,138.0,64.5,0.950088,0.502881
8,Female,92,90,98,175.0,66.0,-0.151357,-1.245288
9,Male,89,93,84,134.0,66.3,-0.103219,-1.057952


In [178]:
#rerun models with the partY2 variable

for val in combination:
    name_var1=str(val[0])
    name_var2=str(val[1])
    name_var3=str(val[2])
    name_var4=str(val[3])
    
    feat_1=data[name_var1]
    feat_2=data[name_var2]
    feat_3=data[name_var3]
    feat_4=data[name_var4]
    
    # Fit the model with the x variable
    model = ols("partY2 ~ feat_1 + feat_2 + feat_3 + feat_4", data).fit()
    
    print(val)
    print(model.summary())
    
    pval=model.pvalues
    
    # Add results <0.05 to list_significant
    for index, value in pval.items():
        if value <= 0.05:
            ind_sig=index
            
            list_ind.append(index)
            list_significant.append(val)
            
            break
            
#report only the models with p<0.05
    
print(list_significant)
print(list_ind)

('Gender', 'FSIQ', 'PIQ', 'Height')
                            OLS Regression Results                            
Dep. Variable:                 partY2   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     2.449
Date:                Mon, 18 May 2020   Prob (F-statistic):             0.0649
Time:                        17:46:10   Log-Likelihood:                -52.032
No. Observations:                  39   AIC:                             114.1
Df Residuals:                      34   BIC:                             122.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercep

In [179]:
for ind in range (len(list_significant)):
    if list_ind[ind]!="Intercept":
        print(list_significant[ind], list_ind[ind])

('Gender', 'FSIQ', 'VIQ', 'PIQ') feat_1[T.Male]
('FSIQ', 'VIQ', 'PIQ', 'Weight') feat_4
('Gender', 'FSIQ', 'PIQ', 'Height') feat_2
('FSIQ', 'VIQ', 'Weight', 'Height') feat_1
('Gender', 'FSIQ', 'VIQ', 'Height') feat_3
('Gender', 'VIQ', 'PIQ', 'Height') feat_2
('FSIQ', 'PIQ', 'Weight', 'Height') feat_1
('Gender', 'FSIQ', 'PIQ', 'Weight') feat_2
('VIQ', 'PIQ', 'Weight', 'Height') feat_1
